In [2]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
#import mysql.connector as connection


In [3]:
#contraseña del servidor
cont = "SQL17Agosto1994"

In [30]:
#funciones que serán utilizadas para la normalización
def cod_cientific(valor):
    if type(valor) != "str" :
        valor = str(valor)
    if (valor.find("E") != -1):
         lista  =valor.split("E")
         num_str = lista[0]
         num_str = num_str.replace(",",".")
         num = float(num_str)
         mult_str = lista[1]
         mult = int(mult_str[1:])
         return (int(num*(10**mult)))
    return int(valor)
def llevar_13(valor):
    if type(valor) != "str" :
        valor = str(valor)
    while len(valor)<13:
        valor = "0" + valor
    return valor
def sacaguion(valor):
    if type(valor) != "str" :
        valor = str(valor)
    if (valor.find("-") != -1):
        return valor.split("-")[2]
    return valor
def redondeo(valor):
    if type(valor) != "float" :
        try:
            valor = float(valor)
        except: 
            return valor
    valor = round(valor,2)
    return valor

In [31]:
#utilizamos los archivos como DataFrames de Pandas
precios_sem1 = pd.read_csv("./Datasets/precios_semana_20200413.csv", sep=",",encoding="UTF-16LE")

In [32]:
precios_sem2 = pd.read_json("./Datasets/precios_semana_20200503.json")

In [33]:
precios_sem3 = pd.read_csv("./Datasets/2precios_semana_20200518 copy.csv", sep="|")

In [34]:
precios_sem4_1 = pd.read_csv("./Datasets/2.1precios_semanas_20200419_20200426.csv",sep=";")

/var/folders/4h/zd1h7yk93j37j92519m4z_y80000gn/T/ipykernel_3239/3651343649.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  precios_sem4_1 = pd.read_csv("./Datasets/2.1precios_semanas_20200419_20200426.csv",sep=";")


In [35]:
precios_sem4_2 =pd.read_csv("./Datasets/2.2precios_semanas_20200419_20200426.csv",sep=";")

/var/folders/4h/zd1h7yk93j37j92519m4z_y80000gn/T/ipykernel_3239/3795902879.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  precios_sem4_2 =pd.read_csv("./Datasets/2.2precios_semanas_20200419_20200426.csv",sep=";")


In [36]:
#definimos la función encargada de normalizar los DataFrames
def CargaDataFrame(DataFrame):
    #primero nos aseguramos que, sea un DataFrame de Pandas, tenga las dimensiones correctas y que sus columnas tengan los nombres correctos (no importa el orden)
    columnas = ['precio', 'sucursal_id', 'producto_id']
    if str(type(DataFrame)) != "<class 'pandas.core.frame.DataFrame'>":
        print("Falla en la carga, inserte en el primer argumento un DataFrame de Pandas")
        return None
    if DataFrame.shape[1] != 3:
        print("Falla en la carga, el DataFrame tiene que poseer 3 columnas (en ese orden): ""precio"", ""producto_id"", ""sucursal_id"" ")
        return None
    if DataFrame.columns[0] not in columnas or DataFrame.columns[1] not in columnas or DataFrame.columns[2] not in columnas:
        print("Los nombres de las columnas son incorrectos, ellos deben ser ", columnas, " respetando las minúsculas")
        return None
    DataFrame = DataFrame[["precio","producto_id", "sucursal_id"]] #acomoda el orden 
    DataFrame.dropna(thresh=3, inplace=True) #eliminamos filas vacías
    DataFrame.dropna(subset="producto_id", inplace=True) #eliminamos las filas que tengan el id de producto vació
    DataFrame["producto_id"] = DataFrame["producto_id"].apply(sacaguion) #sacamos el guion y nos quedamos con el valor de 13 dígitos
    DataFrame["producto_id"] = DataFrame["producto_id"].apply(cod_cientific) #procesamos los que tienen notación científica
    DataFrame["precio"] = DataFrame["precio"].apply(redondeo) #redondeamos el precio a dos decimales
    DataFrame["producto_id"] = DataFrame["producto_id"].apply(llevar_13) #agregamos ceros delante de los Id para que tengan 13 dígitos como el código de barras
    DataFrame["sucursal_id"] = DataFrame["sucursal_id"].replace(to_replace="/",value="-",regex=True) #Cambiamos las barras laterales para igualar todos los id de sucursal
    DataFrame.drop_duplicates(inplace=True) #eliminamos filas duplicadas
    print("El DataFrame fue normalizado correctamente")
    #imprimimos las primers 5 filas para mostrar al usuario
    print(DataFrame.head())
    #Guardamos el DataFrame en nuestra base de datos en MySQL
    cadena_conexion = "mysql+pymysql://root:" + cont + "@localhost:3306/P1_v1"
    conexion = create_engine(cadena_conexion)
    DataFrame.to_sql(name='Precio_semana', con = conexion, if_exists= "append", index=False) #index le indicamos que no esta indexado 
    print("Los archivos pertenecientes al DataFrame se cargaron correctamente")
    return(DataFrame)

Aplicamos la función a los DataFrames

In [37]:
sem2 = CargaDataFrame(precios_sem2)

El DataFrame fue normalizado correctamente
   precio    producto_id sucursal_id
0    29.9  0000000002288     2-1-187
1    39.9  0000000002288     2-3-247
2  499.99  0000000205870     9-1-685
3  539.99  0000000205870      9-2-22
4  519.99  0000000205870      9-2-59
Los archivos pertenecientes al DataFrame se cargaron correctamente


In [38]:
sem1 = CargaDataFrame(precios_sem1)

El DataFrame fue normalizado correctamente
   precio    producto_id sucursal_id
0   29.90  0000000001663     2-1-014
1   29.90  0000000002288     2-1-032
2   39.90  0000000002288     2-1-096
3  499.99  0000000205870     9-1-686
4  519.99  0000000205870     9-2-248
Los archivos pertenecientes al DataFrame se cargaron correctamente


In [39]:
sem3 = CargaDataFrame(precios_sem3)

El DataFrame fue normalizado correctamente
   precio    producto_id sucursal_id
0   29.90  0000000002288     2-1-009
1   32.90  0000000002288     2-1-037
2   36.90  0000000002288     2-1-090
3   39.90  0000000002288     2-3-247
4  499.99  0000000205870     9-1-430
Los archivos pertenecientes al DataFrame se cargaron correctamente


In [40]:
sem4_1 = CargaDataFrame(precios_sem4_1)

/var/folders/4h/zd1h7yk93j37j92519m4z_y80000gn/T/ipykernel_3239/907726508.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataFrame.dropna(thresh=3, inplace=True) #eliminamos filas vacías
/var/folders/4h/zd1h7yk93j37j92519m4z_y80000gn/T/ipykernel_3239/907726508.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataFrame.dropna(subset="producto_id", inplace=True) #eliminamos las filas que tengan el id de producto vació
/var/folders/4h/zd1h7yk93j37j92519m4z_y80000gn/T/ipykernel_3239/907726508.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

El DataFrame fue normalizado correctamente
    precio    producto_id sucursal_id
0    399.0  0000000002288     2-1-092
1    299.0  0000000002288     2-1-206
2    399.0  0000000002288     2-2-241
3  49999.0  0000000205870     9-1-430
4  53999.0  0000000205870       9-2-4
Los archivos pertenecientes al DataFrame se cargaron correctamente


In [41]:
sem4_2 = CargaDataFrame(precios_sem4_2)

/var/folders/4h/zd1h7yk93j37j92519m4z_y80000gn/T/ipykernel_3239/907726508.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataFrame.dropna(thresh=3, inplace=True) #eliminamos filas vacías
/var/folders/4h/zd1h7yk93j37j92519m4z_y80000gn/T/ipykernel_3239/907726508.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataFrame.dropna(subset="producto_id", inplace=True) #eliminamos las filas que tengan el id de producto vació
/var/folders/4h/zd1h7yk93j37j92519m4z_y80000gn/T/ipykernel_3239/907726508.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

El DataFrame fue normalizado correctamente
   precio    producto_id sucursal_id
0    29.9  0000000002288     2-1-184
1    39.9  0000000002288     2-1-206
2  499.99  0000000205870     9-1-430
3  539.99  0000000205870     9-2-107
4  539.99  0000000205870      9-3-18
Los archivos pertenecientes al DataFrame se cargaron correctamente
